In [ ]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# PU
fName1='/eos/uscms/store/user/sghosh/ISOTRACK/MUON/Validation_PU.root'
tree1 = uproot.open(fName1,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))['hcalHBHEMuonHighEta/HBHEMuonHighEta']

In [ ]:
#no PU
fName2='/eos/uscms/store/user/sghosh/ISOTRACK/MUON/Validation_noPU.root'
tree2 = uproot.open(fName2,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))['hcalHBHEMuonHighEta/HBHEMuonHighEta']

In [ ]:
print (tree1.numentries)
print (tree2.numentries)

In [ ]:
branchespu = [b'Event_No',b'pt_of_muon',b'eta_of_muon',b'phi_of_muon', b'energy_of_muon',b'hcal_ieta', 
              b'hcal_iphi',b'MediumMuon',
              b'IsolationR04', b'IsolationR03', b'ecal_3into3', b'hcal_3into3',b'ecal_3x3', b'ho_3into3',
              b'hcal_1x1',
              b'emaxNearP',b'hcal_edepth1',b'hcal_edepth2',b'hcal_edepth3',b'hcal_edepth4',b'hcal_edepth5',
              b'hcal_edepth6',
              b'hcal_edepth7',
              ]

branchesnpu = [b'Event_No',b'hcal_ieta', b'hcal_iphi',
              b'hcal_edepth1',b'hcal_edepth2',b'hcal_edepth3',b'hcal_edepth4',b'hcal_edepth5',
              b'hcal_edepth6',b'hcal_edepth7',
              ]
#dictn = tree.arrays(branches=branches,entrystart=0, entrystop=300)
dictpu = tree1.arrays(branches=branchespu)
dictnpu = tree2.arrays(branches=branchesnpu)

In [ ]:
# function to flatten arrays 
# each event is a row ----> each muon is a row
def flattenarray(dictpu):
    newdict = {}
    for k,v in dictpu.items():
    #    print(k,v.flatten)
       if  b'Event_No' not in k:
            newdict[k] = v.flatten()

    evtl = []
    for i in (range((dictpu[b'Event_No'].shape[0]))):
        toten = len(dictpu[b'hcal_ieta'][i])
        for j in range(toten):
           evtl.append(dictpu[b'Event_No'][i])
        
    newdict[b'Event_No'] = np.array(evtl)
    return newdict

In [ ]:
# flatten -> dataframe
dictpuflat = flattenarray(dictpu)
dictnpuflat = flattenarray(dictnpu)
dfspu = pd.DataFrame.from_dict(dictpuflat)
dfsnpu = pd.DataFrame.from_dict(dictnpuflat)

In [ ]:
# find same muon from both files
merged = pd.merge(dfspu, dfsnpu , on=[b'Event_No',b'hcal_ieta', b'hcal_iphi'])
print ("selected events before cut:",merged.shape[0])
cuts = (merged[b'MediumMuon'])
merged=merged.loc[cuts]
merged = merged.reset_index(drop=True)
print ("selected events after cut:",merged.shape[0])


In [ ]:
# which vars to keep in df
keepvars = [b'pt_of_muon',b'eta_of_muon',b'phi_of_muon', b'energy_of_muon',b'hcal_ieta', 
            b'hcal_iphi',
            b'IsolationR04', b'IsolationR03', b'ecal_3into3', b'hcal_3into3',
            b'ecal_3x3',b'hcal_1x1',
              
            "b'hcal_edepth1'_x","b'hcal_edepth2'_x","b'hcal_edepth3'_x","b'hcal_edepth4'_x",
            "b'hcal_edepth5'_x","b'hcal_edepth6'_x","b'hcal_edepth7'_x",
            "b'hcal_edepth1'_y","b'hcal_edepth2'_y","b'hcal_edepth3'_y","b'hcal_edepth4'_y",
            "b'hcal_edepth5'_y","b'hcal_edepth6'_y","b'hcal_edepth7'_y"
              ]
final_df = merged[keepvars]

In [ ]:
# save dataframe
final_df.to_pickle("test.pkl")